In [22]:
num_nodes = 20
margin = 10 # time margin for arrival before appointments
percentage_of_appointments = .4
GlobalSpanCostCoefficient = 5 # adjust
slack = 200 # adjust
min_work_days = 7 # minimum number of work days per location

work_schedule = {
    1: [9 * 60, 17 * 60],
    2: [9 * 60, 17 * 60],
    3: [9 * 60, 17 * 60],
    4: [7 * 60, 18 * 60],
    5: [8 * 60, 17 * 60],
    6: [8 * 60, 16 * 60],
    7: [8 * 60, 16 * 60]
}
# week (mon-sun) constraints
max_days_off = 0
days_off = {5,6,7,4} # default Sunday no work
max_overnight_stays = 2
no_overnight_stays = {}
if len(no_overnight_stays) > 0:
    possible_overnight_stays = set(range(1, 8)) - no_overnight_stays
else:
    possible_overnight_stays = set(range(1, 8))
max_short_days = 0
short_days = {day for day, hours in work_schedule.items() if (hours[1] - hours[0]) / 60 <= 4}

# Check for short days or overnight stays defined on days off
removed_days = [day for day in list(days_off) if day in short_days]
for day in removed_days:
    days_off.remove(day)
    print(f"Removed day {day} from days off because it's a short day or overnight stay.")

# Check for more days off defined than max days off
if len(days_off) > max_days_off:
    max_days_off = len(days_off)
    print(f"Adjusted max days off to {max_days_off} because {len(days_off)} days off were defined.")

# Check for more short days defined than max short days
if len(short_days) > max_short_days:
    max_short_days = len(short_days)
    print(f"Adjusted max short days to {max_short_days} because {len(short_days)} short days were defined.")

Adjusted max days off to 4 because 4 days off were defined.


In [23]:
penalty_factor = [5, 10000, 500]

from src.routing import solve, create_data
nodes_df, time_matrix = create_data(min_work_days, days_off, percentage_of_appointments, num_nodes, penalty_factor)
nodes_df[['weekday_fixed_appointment', 'priority']]

,weekday_fixed_appointment,priority
0,NaN,916
1,NaN,503
2,NaN,941
3,NaN,500
4,NaN,500
5,3.0,10500
6,NaN,636
7,NaN,500
8,2.0,10500
9,NaN,916


In [24]:
routes_df = solve(nodes_df, time_matrix, slack, days_off, max_days_off, short_days, max_short_days, no_overnight_stays, max_overnight_stays, work_schedule, margin, lunch_duration=30, optimization_algorithm='PARALLEL_CHEAPEST_INSERTION', GlobalSpanCostCoefficient=GlobalSpanCostCoefficient, verbose=False, visual=False, restrictive=False, deep=False)
routes_df[0]['obj_value']

dropped nodes: [15] of which with fixed appointments: [15]
PROOF:     weekday_fixed_appointment cluster
15                        2.0     {3}
dropped nodes: [3, 4, 5] of which with fixed appointments: [5]
PROOF:    weekday_fixed_appointment cluster
5                        3.0     {1}
dropped nodes: [14] of which with fixed appointments: [14]
PROOF:     weekday_fixed_appointment cluster
14                        1.0  {2, 3}
dropped nodes: [5] of which with fixed appointments: [5]
PROOF:    weekday_fixed_appointment cluster
5                        3.0  {1, 2}


0     28426.0
1     76952.0
2     95140.0
3    141960.0
Name: obj_value, dtype: float64